In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import  train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier


from sklearn.metrics import (confusion_matrix, classification_report, \
                             accuracy_score , precision_score, recall_score , f1_score, 
                             roc_curve, roc_auc_score, \
                             log_loss, balanced_accuracy_score)


In [2]:
df1 = pd.read_excel(r"C:\Users\DEVIKA\OneDrive\Desktop\My Learning\DATA STORE\financial data\credit_risk_file_1.xlsx")
df2 = pd.read_excel(r"C:\Users\DEVIKA\OneDrive\Desktop\My Learning\DATA STORE\financial data\credit_risk_file_2.xlsx")

In [3]:
df1.shape, df2.shape

((51336, 26), (51336, 62))

In [4]:
df1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL_months,Age_Newest_TL months
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [5]:
df2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [6]:
df1['PROSPECTID'].max(),df2['PROSPECTID'].max()

(51336, 51336)

In [7]:
df1['PROSPECTID'].duplicated().sum(), df2['PROSPECTID'].duplicated().sum()

(0, 0)

In [8]:
df1['PROSPECTID'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 51336 entries, 0 to 51335
Series name: PROSPECTID
Non-Null Count  Dtype
--------------  -----
51336 non-null  int64
dtypes: int64(1)
memory usage: 401.2 KB


In [9]:
df2['PROSPECTID'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 51336 entries, 0 to 51335
Series name: PROSPECTID
Non-Null Count  Dtype
--------------  -----
51336 non-null  int64
dtypes: int64(1)
memory usage: 401.2 KB


In [10]:
df_merged = pd.merge(df1,df2,on = 'PROSPECTID' , how = 'inner')

In [11]:
df_merged.shape

(51336, 87)

In [12]:
df_merged.info()

# 51,336 samples vs 87 features is a good ratio (≈ 590 samples per feature ratio), which is statistically healthy and safe from COD.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51336 entries, 0 to 51335
Data columns (total 87 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   Total_TL                      51336 non-null  int64  
 2   Tot_Closed_TL                 51336 non-null  int64  
 3   Tot_Active_TL                 51336 non-null  int64  
 4   Total_TL_opened_L6M           51336 non-null  int64  
 5   Tot_TL_closed_L6M             51336 non-null  int64  
 6   pct_tl_open_L6M               51336 non-null  float64
 7   pct_tl_closed_L6M             51336 non-null  float64
 8   pct_active_tl                 51336 non-null  float64
 9   pct_closed_tl                 51336 non-null  float64
 10  Total_TL_opened_L12M          51336 non-null  int64  
 11  Tot_TL_closed_L12M            51336 non-null  int64  
 12  pct_tl_open_L12M              51336 non-null  float64
 13  p

In [13]:
df_merged['Approved_Flag'] = np.where(df_merged['Approved_Flag'] == 'P4',1,0)

In [14]:
df_merged['Approved_Flag'].value_counts(normalize = True)

0    0.885422
1    0.114578
Name: Approved_Flag, dtype: float64

In [15]:
df_merged[df_merged['Tot_Missed_Pmnt(DPD)'] == 23]

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
26583,26584,74,50,24,27,7,0.365,0.095,0.324,0.676,...,0.0,0.0,0.0,57.143,1,0,ConsumerLoan,ConsumerLoan,726,0


In [16]:
df_merged.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,0
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,0
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,0
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0.0,0.0,0.0,9.900,0,0,others,others,673,0
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,0


In [17]:
import pandas as pd
import numpy as np

def drop_highly_correlated(df, target, threshold=0.9):
    """
    Drops highly correlated columns from a DataFrame while keeping
    the feature with the higher correlation with the target.

    Parameters:
    -----------
    df : pandas.DataFrame
        The input dataframe.
    target : str
        Name of the target column.
    threshold : float
        Correlation threshold above which variables will be dropped.

    Returns:
    --------
    pandas.DataFrame
        DataFrame with highly correlated columns removed.
    list
        List of dropped column names.
    """
    corr_matrix = df.corr().abs()

    # Target correlations (excluding the target itself)
    target_corr = corr_matrix[target].drop(target)

    # Only consider feature-feature correlations
    upper_triangle = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )

    to_drop = set()

    for col in upper_triangle.columns:
        for row in upper_triangle.index:
            if col != row and upper_triangle.loc[row, col] > threshold:
                # Compare correlation with target
                if target_corr.get(col, 0) >= target_corr.get(row, 0):
                    to_drop.add(row)
                else:
                    to_drop.add(col)

    reduced_df = df.drop(columns=list(to_drop))
    return reduced_df, list(to_drop)



df_new, dropped = drop_highly_correlated(df_merged, target='Approved_Flag', threshold=0.8)

print("Dropped columns:", dropped)
print(df_new)


Dropped columns: ['num_std', 'num_lss_6mts', 'CC_enq', 'time_since_recent_enq', 'CC_enq_L12m', 'enq_L12m', 'Age_Oldest_TL_months', 'enq_L3m', 'PL_enq_L6m', 'Unsecured_TL', 'max_deliq_6mts', 'time_since_first_deliquency', 'pct_CC_enq_L6m_of_ever', 'pct_closed_tl', 'pct_active_tl', 'time_since_recent_deliquency', 'num_times_60p_dpd', 'pct_opened_TLs_L6m_of_L12m', 'Tot_TL_closed_L12M', 'pct_PL_enq_L6m_of_L12m', 'CC_Flag', 'PL_enq_L12m', 'max_recent_level_of_deliq', 'PL_utilization', 'GL_Flag', 'Total_TL', 'Gold_TL', 'Total_TL_opened_L12M', 'PL_enq', 'CC_enq_L6m', 'num_times_30p_dpd', 'num_std_6mts', 'Tot_Closed_TL', 'num_dbt_6mts', 'num_deliq_12mts', 'num_deliq_6mts', 'enq_L6m']
       PROSPECTID  Tot_Active_TL  Total_TL_opened_L6M  Tot_TL_closed_L6M  \
0               1              1                    0                  0   
1               2              1                    0                  0   
2               3              8                    1                  0   
3          

In [18]:
df_new.columns

Index(['PROSPECTID', 'Tot_Active_TL', 'Total_TL_opened_L6M',
       'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M',
       'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt(DPD)',
       'Auto_TL', 'CC_TL', 'Consumer_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Other_TL', 'Age_Newest_TL months', 'time_since_recent_payment',
       'num_times_delinquent', 'max_delinquency_level', 'num_deliq_6_12mts',
       'max_deliq_12mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts',
       'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'num_lss_12mts',
       'recent_level_of_deliq', 'tot_enq', 'MARITALSTATUS', 'EDUCATION', 'AGE',
       'GENDER', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr',
       'pct_of_active_TLs_ever', 'pct_currentBal_all_TL', 'CC_utilization',
       'PL_Flag', 'pct_CC_enq_L6m_of_L12m', 'pct_PL_enq_L6m_of_ever',
       'max_unsec_exposure_inPct', 'HL_Flag', 'last_prod_enq2',
       'first_prod_enq2', 'Credit_Score', 'Approved_Flag'],
      dtype='

In [19]:
df_new.head(2)

,PROSPECTID,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt(DPD),Auto_TL,...,CC_utilization,PL_Flag,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,1,0,0,0.0,0.0,0.0,0.0,0,0,...,-99999.0,1,0.0,0.0,13.333,1,PL,PL,696,0
1,2,1,0,0,0.0,0.0,1.0,0.0,0,0,...,-99999.0,0,0.0,0.0,0.860,0,ConsumerLoan,ConsumerLoan,685,0


In [20]:
df_new.replace(-99999,0, inplace=True)

In [21]:
df_new.drop(columns=['last_prod_enq2','first_prod_enq2'], inplace=True)

In [22]:
df_new.dtypes

PROSPECTID                     int64
Tot_Active_TL                  int64
Total_TL_opened_L6M            int64
Tot_TL_closed_L6M              int64
pct_tl_open_L6M              float64
pct_tl_closed_L6M            float64
pct_tl_open_L12M             float64
pct_tl_closed_L12M           float64
Tot_Missed_Pmnt(DPD)           int64
Auto_TL                        int64
CC_TL                          int64
Consumer_TL                    int64
Home_TL                        int64
PL_TL                          int64
Secured_TL                     int64
Other_TL                       int64
Age_Newest_TL months           int64
time_since_recent_payment      int64
num_times_delinquent           int64
max_delinquency_level          int64
num_deliq_6_12mts              int64
max_deliq_12mts                int64
num_std_12mts                  int64
num_sub                        int64
num_sub_6mts                   int64
num_sub_12mts                  int64
num_dbt                        int64
n

In [23]:
df_new = df_new[['Tot_Active_TL', 'Total_TL_opened_L6M',
       'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M',
       'pct_tl_open_L12M', 'pct_tl_closed_L12M',
       'Auto_TL', 'CC_TL', 'Consumer_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Other_TL', 'Age_Newest_TL months', 'time_since_recent_payment',
       'num_times_delinquent', 'max_delinquency_level', 'num_deliq_6_12mts',
       'max_deliq_12mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts',
       'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'num_lss_12mts',
       'recent_level_of_deliq', 'tot_enq','Time_With_Curr_Empr',
       'pct_of_active_TLs_ever', 'pct_currentBal_all_TL', 'CC_utilization',
       'PL_Flag', 'pct_CC_enq_L6m_of_L12m', 'pct_PL_enq_L6m_of_ever',
       'max_unsec_exposure_inPct', 'HL_Flag',
       'Approved_Flag']]  #'Credit_Score'

In [25]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import ParameterGrid

def run_xgboost_with_tuning(df_new, target_col, num_features=16, max_diff=0.02, 
                            max_iterations=50, random_state=42):

    np.random.seed(random_state)
    best_result = None  # store best model
    
    # Hyperparameter search space
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
    params_list = list(ParameterGrid(param_grid))

    for iteration in range(max_iterations):
        # Randomly select features
        selected_features = np.random.choice(
            [col for col in df_new.columns if col != target_col],
            num_features,
            replace=False
        )

        # Prepare data
        X = df_new[selected_features]
        y = df_new[target_col]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=random_state
        )

        for params in params_list:
            model = XGBClassifier(random_state=random_state, eval_metric='logloss', **params)
            model.fit(X_train, y_train)

            # Train AUC
            y_train_proba = model.predict_proba(X_train)[:, 1]
            train_auc = roc_auc_score(y_train, y_train_proba)

            # Test AUC
            y_test_proba = model.predict_proba(X_test)[:, 1]
            test_auc = roc_auc_score(y_test, y_test_proba)

            diff = abs(train_auc - test_auc)

            if diff <= max_diff:
                if (best_result is None) or (test_auc > best_result['test_auc']):
                    best_result = {
                        'train_auc': train_auc,
                        'test_auc': test_auc,
                        'features': selected_features,
                        'params': params
                    }
        
        # Stop early if we found a good one
        if best_result is not None and best_result['test_auc'] >= 0.90:
            break

    return best_result


# Example usage
# df_new = your dataframe here
# result = run_xgboost_with_tuning(df_new, target_col='Target')
# print("Best Parameters:", result['params'])
# print("Best Features:", result['features'])
# print(f"Train AUC: {result['train_auc']:.4f}, Test AUC: {result['test_auc']:.4f}")


In [93]:
run_xgboost_with_tuning(df_new, target_col='Approved_Flag', num_features=16, max_diff=0.03, 
                            max_iterations=50, random_state=42)

{'train_auc': 0.9319512945305054,
 'test_auc': 0.9020346940374442,
 'features': array(['Other_TL', 'pct_tl_closed_L6M', 'tot_enq', 'num_dbt',
        'Consumer_TL', 'PL_Flag', 'Age_Newest_TL months',
        'num_times_delinquent', 'CC_utilization', 'Secured_TL',
        'num_std_12mts', 'PL_TL', 'HL_Flag', 'num_sub_12mts',
        'max_delinquency_level', 'Total_TL_opened_L6M'], dtype='<U25'),
 'params': {'colsample_bytree': 0.8,
  'learning_rate': 0.1,
  'max_depth': 5,
  'n_estimators': 150,
  'subsample': 0.8}}

In [26]:
run_xgboost_with_tuning(df_new, target_col='Approved_Flag', num_features=16, max_diff=0.02, 
                            max_iterations=50, random_state=42)

{'train_auc': 0.9199280683982591,
 'test_auc': 0.9003244892571441,
 'features': array(['Other_TL', 'pct_tl_closed_L6M', 'tot_enq', 'num_dbt',
        'Consumer_TL', 'PL_Flag', 'Age_Newest_TL months',
        'num_times_delinquent', 'CC_utilization', 'Secured_TL',
        'num_std_12mts', 'PL_TL', 'HL_Flag', 'num_sub_12mts',
        'max_delinquency_level', 'Total_TL_opened_L6M'], dtype='<U25'),
 'params': {'colsample_bytree': 1.0,
  'learning_rate': 0.2,
  'max_depth': 3,
  'n_estimators': 150,
  'subsample': 0.8}}

In [39]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import roc_auc_score

def run_xgboost_with_constraints(df_new, target_col, num_features=16, max_diff=0.03,
                                 max_auc=0.88, max_iterations=100, random_state=42):
    np.random.seed(random_state)
    best_result = None
    
    # Hyperparameter search space (with regularization)
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.6, 0.8, 1],
        'colsample_bytree': [0.6, 0.8],
        'min_child_weight': [5, 10],     # Higher = less complex
        'gamma': [1, 5, 10],                 # Higher = more conservative
        'reg_lambda': [50, 100, 200,500],           # L2 regularization
        'reg_alpha': [50, 100,200,500]              # L1 regularization
    }
    params_list = list(ParameterGrid(param_grid))

    for iteration in range(max_iterations):
        # Randomly select features
        selected_features = np.random.choice(
            [col for col in df_new.columns if col != target_col],
            num_features,
            replace=False
        )

        # Prepare data
        X = df_new[selected_features]
        y = df_new[target_col]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

        for params in params_list:
            model = XGBClassifier(random_state=random_state, eval_metric='logloss', **params)
            model.fit(X_train, y_train)

            # Train AUC
            y_train_proba = model.predict_proba(X_train)[:, 1]
            train_auc = roc_auc_score(y_train, y_train_proba)

            # Test AUC
            y_test_proba = model.predict_proba(X_test)[:, 1]
            test_auc = roc_auc_score(y_test, y_test_proba)

            diff = abs(train_auc - test_auc)

            # Check constraints
            if (train_auc <= max_auc) and (test_auc <= max_auc) and (diff <= max_diff):
                if (best_result is None) or (test_auc > best_result['test_auc']):
                    best_result = {
                        'train_auc': train_auc,
                        'test_auc': test_auc,
                        'features': selected_features,
                        'params': params
                    }

        # Optional early stop if we find something close to 0.88
        if best_result is not None and best_result['test_auc'] >= max_auc - 0.005:
            break

    return best_result


# Example usage:
# df_new = your dataframe here
# result = run_xgboost_with_constraints(df_new, target_col='Target')
# print("Best Parameters:", result['params'])
# print("Best Features:", result['features'])
# print(f"Train AUC: {result['train_auc']:.4f}, Test AUC: {result['test_auc']:.4f}")


In [41]:
run_xgboost_with_constraints(df_new, target_col='Approved_Flag', num_features=14, max_diff=0.02, 
                            max_iterations=100, random_state=42)

KeyboardInterrupt: 